In [34]:
import numpy as np
from typing import Union
import math

In [206]:
"""
Given an integer B as represented by two arrays a, b
where B = the product over i of (a_i ^ b_i)
return the number of positive integers 
x less than B such that gcd(x, B) = 1
"""
def phi(a: np.array, b: np.array) -> int:
    
    result = 1
    
    for prime,power in zip(a,b):
        
        if power != 0:
        
            result *= (prime**(power-1)) * (prime - 1)
            
    return result

In [207]:
phi([2,3,5,7,11,13], [1,0,0,1,0,0])

6

In [208]:
"""
Given a factor base `factor_base` and an integer a, check whether or not 
a is B-smooth

returns a list `exponents` where each element 
exponents[i] is the exponent of the prime factor_base[i] 
if a is B-smooth

returns an empty array otherwise
"""

def is_B_smooth(a: int, factor_base: np.array) -> Union[np.array, bool]:
    
    exponents = np.zeros_like(factor_base)
    
    for i,p in enumerate(factor_base):
        
        if a == 1:          
            break
        
        while a % p == 0:         
            a = a / p           
            exponents[i] += 1
        
    if a > 1:
        return np.array([])
    
    return exponents

In [210]:
'''
Given a vector of exponents, return the vector mod-2 
'''
def exponent_vector_mod_2(exponents: np.array) -> np.array:
    
    return exponents % 2

In [212]:
"""
Given number to factor N pick a B value for factor base
Taken from paper linked in the chat
"""

def choose_B(N: int) -> int:
    L = np.e ** (0.5 * (np.log(N) * np.log(np.log(N)))**0.5)
    return int(np.ceil(L))
    

In [213]:
"""
Given an integer B, use the sieve of Eratosthenes to
find its prime factors 

returns a list of prime factors less than B

"""

def create_factors(n):
    flags = np.ones(n, dtype=bool)
    flags[0] = flags[1] = False
    for i in range(2, int(np.sqrt(n)) + 1):
        if flags[i]:
            flags[i*i::i] = False
    return np.flatnonzero(flags)

In [214]:
B = choose_B(16921456439215439701)

B

651

In [215]:
create_factors(B)

array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
       317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397,
       401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467,
       479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569,
       571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643,
       647])

In [216]:
from numpy.linalg import svd

def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

In [328]:
def gaussian_elimination_mod2(M):

    contains_pivot = [False]*len(M)
    pivot_found = False
    pivot_loc = {}
    
    for j in range(len(M[0])):
        pivot_found = False
        #Look for pivot
        for i in range(len(M)):
            # Pivot Found at row i and column j
            if(M[i][j] == 1):
                contains_pivot[i] = True
                pivot_loc[j]=i
                pivot_found = True
                break
          
        if pivot_found:
            for k in range(len(M[0])):
                if (k == j):
                    continue
                if (M[i][k] == 1):
                    for l in range(len(M)):
                        M[l][k] = (M[l][j] + M[l][k])%2
                        
    return M, contains_pivot, pivot_loc

# P3(b) find the left null space of M mod 2
def left_null_space_mod2(M): 
    # solve with gaussian elimination
    M, contains_pivot, pivot_loc = gaussian_elimination_mod2(M)
    result = []
    # Find left null space
    for i in range(len(M)):
        #Find dependent rows
        null_space = []
        if not contains_pivot[i]:
            row = M[i]
            null_space = [i]
            for j in range(len(row)):
                if (M[i][j]==1):
                    null_space.append(pivot_loc[j])
            result.append(null_space)
    # Encoding the indicies
    out_vecs = []
    for space in result:
        vec = [0]*len(M)
        for idx in space:
            vec[idx] = 1
        out_vecs.append(vec)
    return out_vecs

ERROR! Session/line number was not unique in database. History logging moved to new session 6


In [326]:
'''
Given an integer n, try to factor it into two large primes
Returns a tuple of the factors if able to factor
Returns false if couldn't factor
''' 
def quadratic_sieve(n: int) -> Union[tuple[int, int], bool]:
    
    
    """
    Choose smoothness bound B
    """
    
    B = choose_B(n)
    factor_base = create_factors(B)
    
    
    # represent B as its prime factorization

    B_primes = factor_base
    B_exp = is_B_smooth(B,factor_base)
    
    if (len(B_exp) == 0):
        # B is a prime itself
        B_primes = [B]
        B_exp = [1]
    

    """
    Find phi(B) + 1 numbers a_i such that 
        b_i = a_i^2 (mod n)
    is B-smooth.
    
    In checking if b_i is B-smooth, we also
    factor b_i.
    
    """
    # assumes B is prime
    phi_B = phi(B_primes, B_exp)
    print(B,phi_B)
    
    all_b_exponents = []
    all_a = []
    all_b = []
    
    curr_a = math.ceil(np.sqrt(n))
    
    while len(all_b_exponents) < phi_B + 1:
        
        # find a new entry to add to the list
        curr_b = curr_a**2 % n
        
        curr_exponents = is_B_smooth(curr_b, factor_base)
        
        if (len(curr_exponents) > 0):

            all_a.append(curr_a)
            all_b.append(curr_b)
            all_b_exponents.append(curr_exponents) # append 0 for -1 factor
        
        curr_a += 1
        
                
    """
    Generate exponent vectors 
    (mod 2) for each b_i
    """
    
    
    all_exponent_vectors = [ exponent_vector_mod_2(b_exponents) for b_exponents in all_b_exponents ]
    
    """
    Find linear combination of these vectors that
    sums to the zero vector
    """
    
    M = np.array(all_exponent_vectors) # matrix of exponent vectors
    
    zero_vector = np.zeros(M.shape[1])
    
#     try:
#         x = np.linalg.solve(M, zero_vector)
#     except np.linalg.LinAlgError:
#         x = np.zeros(M.shape[0])
#         x[0] = 1
    
    
    
    # solutions = nullspace(M.T)
#    solutions *= np.min(solutions)
    solutions = left_null_space_mod2(M)
    """
    Multiply the corresponding a_i for the above 
    combination and call the result (mod n) 'a'
    
    Multiply the corresponding b_i for the above 
    combination and call the result (mod n) 'b'
    
    """
    tol = 1e-13
    for j in solutions:
        print("\n",j)
        x = 1
        y = 1
        for i, k in enumerate(j):
            
            if (k > tol):
                x *= all_a[i]
                y *= all_b[i]
                
                print(all_a[i],all_b[i])
        x = x % n
        y = int(np.sqrt(y)) % n
        factor = np.gcd(x-y, n)
        if (factor != 1 and factor != n):
            return factor, int(n / factor)

    return 
    
    
    
    

ERROR! Session/line number was not unique in database. History logging moved to new session 5


In [ ]:
quadratic_sieve(539873)

quadratic_sieve(16921456439215439701)

19 18

 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
735 352
801 101728
651 360

 [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0